# Set up for dataset and model

Package installation, loading, and dataloaders. There's also a resnet18 model defined.

In [ ]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


# Dataloaders
train_dataset = datasets.CIFAR10('cifar10_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.CIFAR10('cifar10_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:

def tp_relu(x, delta=1.):
    ind1 = (x < -1. * delta).float()
    ind2 = (x > delta).float()
    return .5 * (x + delta) * (1 - ind1) * (1 - ind2) + x * ind2


def tp_smoothed_relu(x, delta=1.):
    ind1 = (x < -1. * delta).float()
    ind2 = (x > delta).float()
    return (x + delta) ** 2 / (4 * delta) * (1 - ind1) * (1 - ind2) + x * ind2


class Normalize(nn.Module):
    def __init__(self, mu, std):
        super(Normalize, self).__init__()
        self.mu, self.std = mu, std

    def forward(self, x):
        return (x - self.mu) / self.std


class IdentityLayer(nn.Module):
    def forward(self, inputs):
        return inputs


class PreActBlock(nn.Module):
    '''Pre-activation version of the BasicBlock.'''
    expansion = 1

    def __init__(self, in_planes, planes, bn, learnable_bn, stride=1, activation='relu'):
        super(PreActBlock, self).__init__()
        self.collect_preact = True
        self.activation = activation
        self.avg_preacts = []
        self.bn1 = nn.BatchNorm2d(
            in_planes, affine=learnable_bn) if bn else IdentityLayer()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=not learnable_bn)
        self.bn2 = nn.BatchNorm2d(
            planes, affine=learnable_bn) if bn else IdentityLayer()
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=not learnable_bn)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=not learnable_bn)
            )

    def act_function(self, preact):
        if self.activation == 'relu':
            act = F.relu(preact)
        elif self.activation[:6] == '3prelu':
            act = tp_relu(preact, delta=float(
                self.activation.split('relu')[1]))
        elif self.activation[:8] == '3psmooth':
            act = tp_smoothed_relu(preact, delta=float(
                self.activation.split('smooth')[1]))
        else:
            assert self.activation[:8] == 'softplus'
            beta = int(self.activation.split('softplus')[1])
            act = F.softplus(preact, beta=beta)
        return act

    def forward(self, x):
        out = self.act_function(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(
            self, 'shortcut') else x  # Important: using out instead of x
        out = self.conv1(out)
        out = self.conv2(self.act_function(self.bn2(out)))
        out += shortcut
        return out


class PreActResNet(nn.Module):
    def __init__(self, block, num_blocks, n_cls, cuda=True, half_prec=False,
                 activation='relu', fts_before_bn=False, normal='none'):
        super(PreActResNet, self).__init__()
        self.bn = True
        self.learnable_bn = True  # doesn't matter if self.bn=False
        self.in_planes = 64
        self.avg_preact = None
        self.activation = activation
        self.fts_before_bn = fts_before_bn
        if normal == 'cifar10':
            self.mu = torch.tensor((0.4914, 0.4822, 0.4465)).view(1, 3, 1, 1)
            self.std = torch.tensor((0.2471, 0.2435, 0.2616)).view(1, 3, 1, 1)
        else:
            self.mu = torch.tensor((0.0, 0.0, 0.0)).view(1, 3, 1, 1)
            self.std = torch.tensor((1.0, 1.0, 1.0)).view(1, 3, 1, 1)
            print('no input normalization')
        if cuda:
            self.mu = self.mu.cuda()
            self.std = self.std.cuda()
        if half_prec:
            self.mu = self.mu.half()
            self.std = self.std.half()

        self.normalize = Normalize(self.mu, self.std)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=not self.learnable_bn)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.bn = nn.BatchNorm2d(512 * block.expansion)
        self.linear = nn.Linear(512*block.expansion, n_cls)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, self.bn,
                          self.learnable_bn, stride, self.activation))
            # layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, return_features=False):
        for layer in [*self.layer1, *self.layer2, *self.layer3, *self.layer4]:
            layer.avg_preacts = []

        out = self.normalize(x)
        out = self.conv1(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        if return_features and self.fts_before_bn:
            return out.view(out.size(0), -1)
        out = F.relu(self.bn(out))
        if return_features:
            return out.view(out.size(0), -1)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out


def PreActResNet18(n_cls, cuda=True, half_prec=False, activation='relu', fts_before_bn=False,
                   normal='none'):
    # print('initializing PA RN-18 with act {}, normal {}'.format())
    return PreActResNet(PreActBlock, [2, 2, 2, 2], n_cls=n_cls, cuda=cuda, half_prec=half_prec,
                        activation=activation, fts_before_bn=fts_before_bn, normal=normal)


# intialize the model
model = PreActResNet18(10, cuda=True, activation='softplus1').to(device)
model.eval()

# Implement the Attacks

Functions are given a simple useful signature that you can start with. Feel free to extend the signature as you see fit.

You may find it useful to create a 'batched' version of PGD that you can use to create the adversarial attack.

In [ ]:
def pgd_linf_untargeted(model, x, labels, k, eps, eps_step):
    model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach()
    adv_x.requires_grad_(True)
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        # TODO: compute the adv_x
        # find delta, clamp with eps
        with torch.no_grad():
            grad_sign = adv_x.grad.sign()
            adv_x = adv_x + eps_step * grad_sign
            delta = torch.clamp(adv_x - x, min=-eps, max=eps)
            adv_x = x + delta
            adv_x = torch.clamp(adv_x, 0.0, 1.0)

    return adv_x

In [ ]:
def pgd_l2_untargeted(model, x, labels, k, eps, eps_step):
    model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach()
    adv_x.requires_grad_(True)
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        batch_size = x.size()[0]
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        grad = adv_x.grad.data
        # TODO: compute the adv_x
        # find delta, clamp with eps, project delta to the l2 ball
        # HINT: https://github.com/Harry24k/adversarial-attacks-pytorch/blob/master/torchattacks/attacks/pgdl2.py
        with torch.no_grad():
            grad = adv_x.grad

            # normalize gradient in L2
            grad_norm = grad.view(grad.size(0), -1).norm(p=2, dim=1)
            grad_norm = grad_norm.view(-1, *([1] * (grad.dim() - 1))) + 1e-10
            
            grad_unit = grad / grad_norm

            # take a step
            adv_x = adv_x + eps_step * grad_unit

            # project back to L2 ball
            delta = adv_x - x
            delta_norm = delta.view(delta.size(0), -1).norm(p=2, dim=1)
            delta_norm = delta_norm.view(-1, *
                                         ([1] * (delta.dim() - 1))) + 1e-10
            delta_norm_clip = torch.clamp(delta_norm, min=eps) / eps   # =1 if ||delta|| <= eps, >1 otherwise
            delta = delta / delta_norm_clip

            adv_x = x + delta
            adv_x = torch.clamp(adv_x, 0.0, 1.0)

    return adv_x

# Evaluate Single and Multi-Norm Robust Accuracy

In this section, we evaluate the model on the Linf and L2 attacks as well as union accuracy.

In [ ]:
def test_model_on_single_attack(model, attack='pgd_linf', eps=0.1):
    model.eval()
    tot_test, tot_acc, tot_clean = 0.0, 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in tqdm(enumerate(test_loader), total=len(test_loader), desc="Evaluating"):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        if attack == 'pgd_linf':
            # TODO: get x_adv untargeted pgd linf with eps, and eps_step=eps/4
            x_adv = pgd_linf_untargeted(model, x_batch, y_batch,
                                        k=10,
                                        eps=eps,
                                        eps_step=eps/4)
        elif attack == 'pgd_l2':
            # TODO: get x_adv untargeted pgd l2 with eps, and eps_step=eps/4
            x_adv = pgd_l2_untargeted(model, x_batch, y_batch,
                                      k=10,
                                      eps=eps,
                                      eps_step=eps/4)
        else:
            pass

        # get the testing accuracy and update tot_test and tot_acc
        with torch.no_grad():
            out = model(x_adv)
            out_clean = model(x_batch)
            preds = out.argmax(dim=1)
            preds_clean = out_clean.argmax(dim=1)
            correct = (preds == y_batch).sum().item()
            correct_clean = (preds_clean == y_batch).sum().item()
        tot_clean += correct_clean
        tot_acc += correct
        tot_test += x_batch.size(0)

    print('Standard accuracy %.5lf' % (tot_clean/tot_test))
    print('Robust accuracy %.5lf' % (tot_acc/tot_test),
          f'on {attack} attack with eps = {eps}')

# PGD Adversarial Training & Evaluation (added)
This section was appended automatically to run PGD adversarial training, compare with standard training, report accuracies across epsilons, and visualize adversarial examples.

In [ ]:
import time
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
def fgsm_attack(model, x, y, eps):
    return pgd_linf_untargeted(model, x, y, k=1, eps=eps, eps_step=eps, random_start=False)


In [ ]:
@torch.no_grad()
def eval_clean_accuracy(model, loader):
    model.eval()
    total=0; correct=0
    for xb,yb in loader:
        xb,yb=xb.to(device), yb.to(device)
        out=model(xb)
        pred=out.argmax(1)
        total+=xb.size(0); correct+=(pred==yb).sum().item()
    return 100.0*correct/total

def eval_robust_accuracy(model, loader, eps=0.1, steps=20):
    model.eval()
    total=0; correct=0
    for xb,yb in loader:
        xb,yb=xb.to(device), yb.to(device)
        xb_adv = pgd_linf_untargeted(model, xb, yb, k=steps, eps=eps, eps_step=eps/4.0, random_start=True)
        with torch.no_grad():
            out = model(xb_adv)
            pred = out.argmax(1)
            total += xb.size(0)
            correct += (pred==yb).sum().item()
    return 100.0*correct/total


In [ ]:
# training helpers: standard and adversarial (on-the-fly PGD)
def standard_train(model, loader, epochs=3, lr=1e-3):
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    ce = nn.CrossEntropyLoss()
    start=time.time()
    for ep in range(epochs):
        model.train()
        running=0.0
        for xb,yb in tqdm(loader, desc=f"Std ep {ep+1}/{epochs}"):
            xb,yb=xb.to(device), yb.to(device)
            opt.zero_grad()
            out=model(xb)
            loss=ce(out,yb)
            loss.backward(); opt.step()
            running+=loss.item()
        print(f"Std epoch {ep+1} avg_loss={running/len(loader):.4f}")
    return time.time()-start

def adversarial_train_pgd(model, loader, epochs=3, eps=8/255, k=10, lr=1e-3):
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    ce = nn.CrossEntropyLoss()
    start=time.time()
    for ep in range(epochs):
        model.train()
        running=0.0
        for xb,yb in tqdm(loader, desc=f"Adv ep {ep+1}/{epochs}"):
            xb,yb=xb.to(device), yb.to(device)
            xb_adv = pgd_linf_untargeted(model, xb, yb, k=k, eps=eps, eps_step=eps/4.0, random_start=True)
            opt.zero_grad()
            out_adv = model(xb_adv)
            out_clean = model(xb)
            loss = 0.5 * ce(out_clean, yb) + 0.5 * ce(out_adv, yb)
            loss.backward(); opt.step()
            running+=loss.item()
        print(f"Adv epoch {ep+1} avg_loss={running/len(loader):.4f}")
    return time.time()-start


In [ ]:
# Run experiments over epsilons 0.01..0.1 (10 values). Uses train_loader/test_loader defined earlier in your notebook.
eps_values = [2/255, 8/255]
results=[]
std_epochs=10
adv_epochs=10

for eps in eps_values:
    print('\n'+'='*50)
    print(f"Running for eps={eps:.3f}")
    m_std = PreActResNet18(10, cuda=True, activation='softplus1').to(device)
    m_adv = PreActResNet18(10, cuda=True, activation='softplus1').to(device)

    # standard train
    t_std = standard_train(m_std, train_loader, epochs=std_epochs)
    clean_std = eval_clean_accuracy(m_std, test_loader)
    rob_std = eval_robust_accuracy(m_std, test_loader, eps=eps, steps=20)
    print(f"Standard: clean={clean_std:.2f}%, PGD({eps:.3f})={rob_std:.2f}%, time={t_std:.1f}s")

    # adversarial train
    t_adv = adversarial_train_pgd(m_adv, train_loader, epochs=adv_epochs, eps=eps, k=10)
    clean_adv = eval_clean_accuracy(m_adv, test_loader)
    rob_adv = eval_robust_accuracy(m_adv, test_loader, eps=eps, steps=20)
    print(f"Adversarial: clean={clean_adv:.2f}%, PGD({eps:.3f})={rob_adv:.2f}%, time={t_adv:.1f}s")

    fgsm_std = eval_robust_accuracy(m_std, test_loader, eps=eps, steps=1)
    fgsm_adv = eval_robust_accuracy(m_adv, test_loader, eps=eps, steps=1)
    print(f"FGSM(k=1) acc: std={fgsm_std:.2f}%, adv={fgsm_adv:.2f}%")


import pandas as pd
df = pd.DataFrame(results)
print('\n=== Summary ===')
display(df)
